In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

22/02/20 21:24:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
df = spark.read.option("header", "true").csv("data/final_dataset.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Flow ID: string (nullable = true)
 |-- Src IP: string (nullable = true)
 |-- Src Port: string (nullable = true)
 |-- Dst IP: string (nullable = true)
 |-- Dst Port: string (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: string (nullable = true)
 |-- Tot Fwd Pkts: string (nullable = true)
 |-- Tot Bwd Pkts: string (nullable = true)
 |-- TotLen Fwd Pkts: string (nullable = true)
 |-- TotLen Bwd Pkts: string (nullable = true)
 |-- Fwd Pkt Len Max: string (nullable = true)
 |-- Fwd Pkt Len Min: string (nullable = true)
 |-- Fwd Pkt Len Mean: string (nullable = true)
 |-- Fwd Pkt Len Std: string (nullable = true)
 |-- Bwd Pkt Len Max: string (nullable = true)
 |-- Bwd Pkt Len Min: string (nullable = true)
 |-- Bwd Pkt Len Mean: string (nullable = true)
 |-- Bwd Pkt Len Std: string (nullable = true)
 |-- Flow Byts/s: string (nullable = true)
 |-- Flow Pkts/s: string (nullable =

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

In [16]:
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType

df = df.withColumn("TotLen Fwd Pkts",col("TotLen Fwd Pkts").cast(FloatType()))
discretizer = QuantileDiscretizer(numBuckets=10, inputCol="TotLen Fwd Pkts", outputCol="totlen_disc")

df = discretizer.fit(df).transform(df)
df.show()

22/02/20 21:31:41 WARN QuantileDiscretizer: Some quantiles were identical. Bucketing to 6 buckets as a result.


+---+--------------------+-------------+--------+------------+--------+--------+--------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+------------------+------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+-----------+-----------------+------------------+-----------+-----------+-----------+-----------------+------------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+------------------+------------------+-----------+-----------+-----------------+------------------+-----------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------------+------------------+------------------+--------------+--------------+----------------+------------

In [19]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.ml import Pipeline 
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.feature import StringIndexer 
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexers = [
StringIndexer(inputCol="Dst IP", outputCol = "dstip_index"),
StringIndexer(inputCol="FIN Flag Cnt", outputCol = "fin_index"),
StringIndexer(inputCol="SYN Flag Cnt", outputCol = "syn_index"),
StringIndexer(inputCol="URG Flag Cnt", outputCol = "urg_index"),
StringIndexer(inputCol="PSH Flag Cnt", outputCol = "cwe_index"),
StringIndexer(inputCol="ECE Flag Cnt", outputCol = "ece_index"),
StringIndexer(inputCol="totlen_disc", outputCol = "totlen_index"),
StringIndexer(inputCol="Label", outputCol = "label")]

In [20]:
pipeline = Pipeline(stages=indexers)
df = df.select("Dst IP", 
               "FIN Flag Cnt", 
               "SYN Flag Cnt", 
               "URG Flag Cnt",
               "PSH Flag Cnt",
               "ECE Flag Cnt",
               "totlen_disc",
               "Label"
              )
#df = df.sample(fraction=0.3)
indexed_rows_df = pipeline.fit(df).transform(df)
indexed_rows_df.show(5, False)

22/02/20 21:43:03 WARN DAGScheduler: Broadcasting large task binary with size 1288.9 KiB


+------------+------------+------------+------------+------------+------------+-----------+-----+-----------+---------+---------+---------+---------+---------+------------+
|Dst IP      |FIN Flag Cnt|SYN Flag Cnt|URG Flag Cnt|PSH Flag Cnt|ECE Flag Cnt|totlen_disc|label|dstip_index|fin_index|syn_index|urg_index|cwe_index|ece_index|totlen_index|
+------------+------------+------------+------------+------------+------------+-----------+-----+-----------+---------+---------+---------+---------+---------+------------+
|203.73.24.75|0           |1           |0           |0           |0           |3.0        |0.0  |25.0       |0.0      |1.0      |0.0      |0.0      |0.0      |2.0         |
|203.73.24.75|0           |0           |0           |0           |0           |1.0        |0.0  |25.0       |0.0      |0.0      |0.0      |0.0      |0.0      |0.0         |
|203.73.24.75|0           |1           |0           |0           |0           |3.0        |0.0  |25.0       |0.0      |1.0      |0.0   

In [21]:
vectorAssembler = VectorAssembler(inputCols = ["dstip_index","fin_index", "syn_index", "urg_index", "cwe_index","ece_index", "totlen_index"],outputCol = "features")
vindexed_rows_df = vectorAssembler.transform(indexed_rows_df)
vindexed_rows_df.show(5, False)

22/02/20 21:46:05 WARN DAGScheduler: Broadcasting large task binary with size 1296.1 KiB


+------------+------------+------------+------------+------------+------------+-----------+-----+-----------+---------+---------+---------+---------+---------+------------+--------------------------+
|Dst IP      |FIN Flag Cnt|SYN Flag Cnt|URG Flag Cnt|PSH Flag Cnt|ECE Flag Cnt|totlen_disc|label|dstip_index|fin_index|syn_index|urg_index|cwe_index|ece_index|totlen_index|features                  |
+------------+------------+------------+------------+------------+------------+-----------+-----+-----------+---------+---------+---------+---------+---------+------------+--------------------------+
|203.73.24.75|0           |1           |0           |0           |0           |3.0        |0.0  |25.0       |0.0      |1.0      |0.0      |0.0      |0.0      |2.0         |(7,[0,2,6],[25.0,1.0,2.0])|
|203.73.24.75|0           |0           |0           |0           |0           |1.0        |0.0  |25.0       |0.0      |0.0      |0.0      |0.0      |0.0      |0.0         |(7,[0],[25.0])            |


In [22]:
splits = vindexed_rows_df.randomSplit([0.6,0.4], 42) 
# optional value 42 is seed for sampling 
train_df = splits[0] 
test_df = splits[1]

In [23]:
nb = NaiveBayes(modelType="multinomial")
nbmodel = nb.fit(train_df)

22/02/20 21:46:11 WARN DAGScheduler: Broadcasting large task binary with size 1331.4 KiB
22/02/20 21:47:36 WARN DAGScheduler: Broadcasting large task binary with size 1306.3 KiB


In [24]:
predictions_df = nbmodel.transform(test_df)
predictions_df.show(5, True)

22/02/20 21:50:27 WARN DAGScheduler: Broadcasting large task binary with size 1335.7 KiB


+------------+------------+------------+------------+------------+------------+-----------+-----+-----------+---------+---------+---------+---------+---------+------------+---------+--------------------+--------------------+----------+
|      Dst IP|FIN Flag Cnt|SYN Flag Cnt|URG Flag Cnt|PSH Flag Cnt|ECE Flag Cnt|totlen_disc|label|dstip_index|fin_index|syn_index|urg_index|cwe_index|ece_index|totlen_index| features|       rawPrediction|         probability|prediction|
+------------+------------+------------+------------+------------+------------+-----------+-----+-----------+---------+---------+---------+---------+---------+------------+---------+--------------------+--------------------+----------+
|172.31.69.25|           0|           0|           0|           0|           0|        1.0|  0.0|        0.0|      0.0|      0.0|      0.0|      0.0|      0.0|         0.0|(7,[],[])|[-0.6812565473673...|[0.50598080390174...|       0.0|
|172.31.69.25|           0|           0|           0|   

In [26]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy") 
nbaccuracy = evaluator.evaluate(predictions_df) 
print("Test accuracy = " + str(nbaccuracy))
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
nbf1 = evaluator.evaluate(predictions_df) 
print("F1 score = " + str(nbf1))

22/02/20 21:57:20 WARN DAGScheduler: Broadcasting large task binary with size 1332.0 KiB


Test accuracy = 0.7113657514191387


22/02/20 21:58:53 WARN DAGScheduler: Broadcasting large task binary with size 1332.0 KiB


F1 score = 0.7072786791719761
